In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import matplotlib.pyplot as plt
import cv2
import numpy as np      
from time import sleep

base = BaseOverlay("base.bit")
print("overlay init")

overlay init


In [2]:
# 硬件模块
sw0 = base.switches[0]
sw1 = base.switches[1]
rgbled_position = [4,5]
print("buttons init")

buttons init


In [ ]:
# 短信模块
import json
from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.common.exception.tencent_cloud_sdk_exception import TencentCloudSDKException
from tencentcloud.sms.v20210111 import sms_client, models
# try:
msg_info = [0, 0, 0] # 控制变量 0代表无 1代表有
def sendmsg(msg_info):
    cred = credential.Credential("XXX", "XXX")
    httpProfile = HttpProfile()
    httpProfile.endpoint = "sms.tencentcloudapi.com"

    clientProfile = ClientProfile()
    clientProfile.httpProfile = httpProfile
    client = sms_client.SmsClient(cred, "ap-nanjing", clientProfile)

    req = models.SendSmsRequest()
    
    msg_send = ['无', '无', '无']
    for i in range(3):
        if msg_info[i] == 1:
            msg_send[i] = '有'
            
    params = {
        "PhoneNumberSet": ["12345678910"],  # 手机号可输入多个，即群发消息
        "SmsSdkAppId": "XX",  # appid，每个账户唯一
        "SignName": "XXX",  # 签名， 每个账户唯一
        "TemplateId": "XXX",  # 正文模板ID，不同模板改不同
        "TemplateParamSet": [ msg_send[0], msg_send[1], msg_send[2] ]
    }
    req.from_json_string(json.dumps(params))

    resp = client.SendSms(req)
    print(resp.to_json_string())
print('def sendmsg init')

In [3]:
# 语音模块
pAudio = base.audio 
pAudio.select_microphone()

def audio_play(str) :
    if str == 'known':
        pAudio.load("/home/xilinx/jupyter_notebooks/face/audio/known.wav")
        pAudio.play()
    elif str == 'unknown':
        pAudio.load("/home/xilinx/jupyter_notebooks/face/audio/unknown.wav")
        pAudio.play()
    elif str == 'fall':
        pAudio.load("/home/xilinx/jupyter_notebooks/face/audio/fall.wav") # 用不了就将face替换base
        pAudio.play()

print("audio init")
# audio_play('fall')

audio init


In [4]:
# 面容识别模块
recognizer = cv2.face.createLBPHFaceRecognizer()
recognizer.load(
    '/home/xilinx/jupyter_notebooks/face/'
    'trainner.yml')
face_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/face/'
    'haarcascade_frontalface_alt.xml')
# 变量定义
idnum = 0
p = [0,0]
count = 0
count1 = 0
# 录入用户编号
names = ['初始', 'zyb', 'jwj', 'user2', 'user3']
print("face recognition init")

face recognition init


In [5]:
# 姿态检测模块
fg = cv2.createBackgroundSubtractorMOG2()  # 实例化高斯混合模型
print("gesture init")

gesture init


In [6]:
# HDMI & 视频模块
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

frame_out_w = 640
frame_out_h = 480

frame_in_w = 640
frame_in_h = 480

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
minW = 0.1 * cap.get(3)
minH = 0.1 * cap.get(4)
print("capture device is open: " + str(cap.isOpened()))

capture device is open: True


In [7]:
for i in range(5)
    cap.read()
base.rgbleds[4].write(0)
base.rgbleds[5].write(0)
for led in base.leds:
    led.off()
print("begin")

while True:
    ret, frame = cap.read()
    
    if sw0.read() == 0:
         # 识别人脸
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray,
                                              scaleFactor=1.2,
                                              minNeighbors=5,
                                              minSize=(int(minW), int(minH)))
        if faces is not None:

            # 进行校验
            for (x, y, w, h) in faces:
                if count1 < 10:
                    count1 = count1 + 1
                count = count + 1
                idnum, confidence = recognizer.predict(gray[y:y + h, x:x + w])
                print(idnum, confidence)
    #             plt.imshow(frame)
    #             plt.show()
                if confidence < 45:
    #                 print("jwj")
                    p[0] = p[0] + 1
                else:
                    p[1] = p[1] + 1

        if count1 < 10:
            continue
    #     print("we start to detect the face")
        if count >= 5:
            if p[0] == 0 and p[1] == 0:
                continue
            elif p[0] != 0:
                print("known")
                base.rgbleds[4].write(1)
                base.rgbleds[5].write(1)
                audio_play('known') 
            else:
                print("unknown")
                base.rgbleds[4].write(6)
                base.rgbleds[5].write(6)
                audio_play('unknown') 
                msg_info[0] = 1
            p[0] = 0
            p[1] = 0
            count = 0
    if sw0.read() == 1:
        blurred = cv2.GaussianBlur(frame, (3, 3), 0)  # 高斯滤波
        gray = cv2.cvtColor(blurred, cv2.COLOR_RGB2GRAY)  # 灰度图
        xgrad = cv2.Sobel(gray, cv2.CV_16SC1, 1, 0)  # x方向梯度
        ygrad = cv2.Sobel(gray, cv2.CV_16SC1, 0, 1)  # y方向梯度
        edge_output = cv2.Canny(xgrad, ygrad, 50, 150)  # canny 边缘检测
        fgmask = fg.apply(edge_output)  # 使用高斯混合模型，背景减除
        
        # 闭运算
        hline = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 4),
                                          (-1, -1))  # 定义结构元素，卷积核
        vline = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 1), (-1, -1))
        result = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, hline)  # 水平方向
        result = cv2.morphologyEx(result, cv2.MORPH_CLOSE, vline)  # 垂直方向
        dilateim = cv2.dilate(result,
                              cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4)),
                              iterations=1)  # 使边缘膨胀
        frame, contours, hierarchy  = cv2.findContours(dilateim, cv2.RETR_EXTERNAL,
                                          cv2.CHAIN_APPROX_NONE)  # 查找轮廓

        for c in contours:
            if cv2.contourArea(c) > 10000:
                (x, y, w, h) = cv2.boundingRect(c)
                scale = w / h

        # 根据比例判断， 首先人不能离摄像头太近，其次可以打印scale来调参
        if scale > 2:
            #print(scale)
            base.rgbleds[4].write(3)
            base.rgbleds[5].write(3)
            for led in base.leds:
                led.on() 
            print('摔倒了')
            audio_play('fall')
            msg_info[1] = 1
        elif 0<scale and scale<2:
            base.rgbleds[4].write(0)
            base.rgbleds[5].write(0)
            for led in base.leds:
                led.off()
            #print(scale)
        
    cv2.waitKey(1)      
        
    # switches开关
    if sw1.read() == 1:
        print("close window")
        sendmsg(msg_info) # 发送短信
        msg_info[0] = 0
        msg_info[1] = 0
        msg_info[2] = 0
        break
        
    

    

base.rgbleds[4].write(0)
base.rgbleds[5].write(0)
cv2.waitKey(1)
cap.release()
print("video over")

begin
2 59.255334293952835
2 62.833593820049785
2 104.131247229869
2 55.825843244475315
2 62.058753436345576
2 95.80876084930313
2 50.402140167442276
2 44.524089284950456
2 49.7057153273638
2 46.76512772170506
known
2 52.09642877488352
2 48.9186102769036
2 95.46354947524719
2 64.42448859083262
2 63.37800047788519
unknown
2 56.70520228442461
2 56.968376519426315
2 52.28149339478327
2 48.38406358518382
2 46.87496271178217
unknown
2 44.41662763024221
2 44.285947156039555
2 45.70115554995634
2 46.6275958039842
2 73.03937054442153
known
2 40.1729053610607
2 40.32032721643686
2 42.916450427192835
2 43.56025679468918
2 43.12786620075237
known
2 42.79392723945359
2 40.37053761868872
2 42.41510866407208
2 37.896828116890894
2 41.40143485937281
known
close window
video over


In [ ]:
#### HDMI 输出部分
outframe = hdmi_out.newframe()
outframe[:] = frame
hdmi_out.writeframe(outframe)